In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('./retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# Оценивание
За выполнени каждого задания 1 балл

4 балла -> отл

3 балла -> хор

И тд

### Задание 0. Товар 999999
На вебинаре мы использовали товар 999999 - что это за товар?  
Зачем он нужен?  
Используя этот товар мы смещаем качество рекомендаций.
В какую сторону?   
Можно ли удалить этот товар?   
Уберите этот товар и сравните с качеством на семинаре.

Качество уменьшается. Это дополнительная обобщающая информация, которую мы убираем, если не указываем
.

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [4]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    weights = np.array(items_weights['weights'])
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

Сделайте предсказания

In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

In [6]:
items = data_train['item_id'].value_counts()
items_logs = np.log(items)
items_weights = items_logs / items_logs.sum()

items_weights = items_weights.reset_index()
items_weights.columns = ['item_id', 'weights']

In [7]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights))
result

CPU times: user 6.18 s, sys: 0 ns, total: 6.18 s
Wall time: 6.18 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12524979, 882620, 830374, 10121521, 1007836]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[885061, 1071698, 276370, 14025461, 865885]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[6001437, 8159433, 1137901, 12300991, 924691]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1961448, 7440665, 958214, 1328148, 5588595]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[2276984, 34214, 15799296, 15926944, 837137]"
...,...,...,...
2037,2496,[6534178],"[674307, 8357617, 925119, 1077514, 1471570]"
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[13876669, 448902, 12677764, 15975136, 12487785]"
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[967166, 13190028, 825290, 15452499, 2432680]"
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[451047, 10311411, 7410362, 901280, 740677]"


In [8]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [9]:
items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

,user_id,actual,weighted_random_recommendation,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12524979, 882620, 830374, 10121521, 1007836]","[89456, 13008073, 1935096, 1066488, 1093200]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[885061, 1071698, 276370, 14025461, 865885]","[13416624, 1038134, 13074773, 8020230, 6772835]"


In [10]:
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [11]:
%%time

# Можно так делать, так как рекомендация не зависит от юзера
popular_recs = popularity_recommendation(data_train, n=5)

result['popular_recommendation'] = result['user_id'].apply(lambda x: popular_recs)
result.head(2)

CPU times: user 162 ms, sys: 0 ns, total: 162 ms
Wall time: 159 ms


,user_id,actual,weighted_random_recommendation,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12524979, 882620, 830374, 10121521, 1007836]","[89456, 13008073, 1935096, 1066488, 1093200]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[885061, 1071698, 276370, 14025461, 865885]","[13416624, 1038134, 13074773, 8020230, 6772835]","[6534178, 6533889, 1029743, 6534166, 1082185]"


In [12]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [13]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [14]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [16]:
model = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

In [17]:
%%time

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 128 ms, sys: 437 µs, total: 129 ms
Wall time: 124 ms


In [18]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 4.19 s, sys: 826 ms, total: 5.01 s
Wall time: 2.06 s


In [19]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 127 ms, sys: 63.6 ms, total: 191 ms
Wall time: 183 ms


In [20]:
%%time

model = CosineRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 3.9 s, sys: 594 ms, total: 4.5 s
Wall time: 1.8 s


In [21]:
%%time

result['cosine'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)])

CPU times: user 158 ms, sys: 0 ns, total: 158 ms
Wall time: 153 ms


In [22]:
%%time

model = TFIDFRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 4.18 s, sys: 601 ms, total: 4.78 s
Wall time: 1.94 s


In [23]:
%%time

result['tfidf'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)])

CPU times: user 137 ms, sys: 16.8 ms, total: 154 ms
Wall time: 133 ms


In [24]:
%%time

model = ItemItemRecommender(K=1, num_threads=4) # K - кол-во билжайших соседей

model.fit(csr_matrix(user_item_matrix).T.tocsr(), 
          show_progress=True)

recs = model.recommend(userid=userid_to_id[1], 
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=False)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 4.32 s, sys: 831 ms, total: 5.16 s
Wall time: 2.12 s


In [25]:
%%time

result['own_purchases'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[999999], 
                                    recalculate_user=False)])

CPU times: user 138 ms, sys: 581 µs, total: 139 ms
Wall time: 128 ms


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [26]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [27]:
def evaluete_pred(data, true):
    for col in data.columns[2:]:
        res = data.apply(lambda row: precision_at_k(row[col], row[true]), axis=1).mean()
        print(f'{col} : {res}')

In [28]:
evaluete_pred(result, 'actual')

weighted_random_recommendation : 0.0002938295788442704
random_recommendation : 0.0002938295788442704
popular_recommendation : 0.15523996082272082
itemitem : 0.13692458374142857
cosine : 0.13290891283055686
tfidf : 0.1389813907933383
own_purchases : 0.17969311132876015


### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


In [29]:
data_test_top_5000 = data_test[data_test['item_id'].isin(top_5000)]
result_top5000 = data_test_top_5000.groupby('user_id')['item_id'].unique().reset_index()
result_top5000.columns=['user_id', 'actual']

In [30]:
items = data_train.item_id.unique()

result_top5000['random_recommendation'] = result_top5000['user_id'].apply(lambda x: random_recommendation(items, n=5))
result_top5000.head()

,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 914190, 95804...","[1015177, 1013321, 5585966, 10254382, 848322]"
1,3,"[851057, 872021, 878302, 879948, 909638, 91320...","[9706982, 1094955, 1134633, 866778, 7166723]"
2,6,"[920308, 926804, 1017061, 1078346, 1120741, 82...","[858302, 934112, 937845, 5591814, 1126252]"
3,7,"[840386, 889774, 898068, 909714, 953476, 97699...","[1119261, 6514160, 1050079, 846634, 1102456]"
4,8,"[835098, 872137, 910439, 924610, 1041259, 5569...","[1054030, 1082185, 839849, 1024032, 1083328]"


In [31]:
%%time

popular_recs = popularity_recommendation(data_train, n=5)

result_top5000['popular_recommendation'] = result_top5000['user_id'].apply(lambda x: popular_recs)
result_top5000.head(2)

CPU times: user 83.7 ms, sys: 10.3 ms, total: 94 ms
Wall time: 89.6 ms


,user_id,actual,random_recommendation,popular_recommendation
0,1,"[821867, 834484, 856942, 865456, 914190, 95804...","[1015177, 1013321, 5585966, 10254382, 848322]","[999999, 6534178, 6533889, 1029743, 6534166]"
1,3,"[851057, 872021, 878302, 879948, 909638, 91320...","[9706982, 1094955, 1134633, 866778, 7166723]","[999999, 6534178, 6533889, 1029743, 6534166]"


In [32]:
result_top5000['weighted_random_recommendation'] = result_top5000['user_id'].apply(lambda x: weighted_random_recommendation(items_weights))
result_top5000.head()

,user_id,actual,random_recommendation,popular_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 914190, 95804...","[1015177, 1013321, 5585966, 10254382, 848322]","[999999, 6534178, 6533889, 1029743, 6534166]","[34180, 1060567, 935389, 960505, 15741936]"
1,3,"[851057, 872021, 878302, 879948, 909638, 91320...","[9706982, 1094955, 1134633, 866778, 7166723]","[999999, 6534178, 6533889, 1029743, 6534166]","[1132282, 848127, 1769038, 1021673, 9858920]"
2,6,"[920308, 926804, 1017061, 1078346, 1120741, 82...","[858302, 934112, 937845, 5591814, 1126252]","[999999, 6534178, 6533889, 1029743, 6534166]","[9880774, 7167540, 8205695, 15452241, 823858]"
3,7,"[840386, 889774, 898068, 909714, 953476, 97699...","[1119261, 6514160, 1050079, 846634, 1102456]","[999999, 6534178, 6533889, 1029743, 6534166]","[874753, 1102554, 8158940, 905771, 1754352]"
4,8,"[835098, 872137, 910439, 924610, 1041259, 5569...","[1054030, 1082185, 839849, 1024032, 1083328]","[999999, 6534178, 6533889, 1029743, 6534166]","[1002076, 13417421, 872531, 13910756, 8293486]"


In [33]:
evaluete_pred(result_top5000, 'actual')

random_recommendation : 0.006382978723404249
popular_recommendation : 0.08855116514690954
weighted_random_recommendation : 0.0005065856129685917


### Задание 4. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.